In [1]:
from stardist.plot import render_label
from csbdeep.utils import normalize
from stardist.models import StarDist2D
import matplotlib.pyplot as plt
import numpy as np
import os
import tifffile as tiff
import skimage.io as io
import skimage.segmentation

## Nuclear Segmentation 

In [2]:
def StarDist_segmentor(in_dir, save_dir):
    '''
    Produces label masks as nuclear segmentation output of StarDist segementation algorithm

    Parameters
    in_dir: str
        the path to the directory containing the grayscaled DAPI channel images of various fields. All images should be in .tiff format
    save_dir_seg: str
        the path to the directory where the segmented images will be saved as masks. Each tiff image in the input directory will
        have a corresponding mask in the directory provided by the user
    '''
    
    # iterating over all files in the input directory
    for file in os.listdir(in_dir):
        # only processes tiff images
        if file[-5:]=='.tiff' or file[-4:]=='.tif':
            # reads the grayscaled DAPI channel of each field as an array
            DAPI_gray = io.imread(os.path.join(in_dir, file), as_gray=True)
            # loads pretrained stardist model
            model = StarDist2D.from_pretrained('2D_versatile_fluo')
            # predicting nuclei using the StarDist model
            normalized_img = normalize(DAPI_gray)
            label_mask, _ = model.predict_instances(normalized_img)
            io.imsave(os.path.join(save_dir, file[:file.find('.')] + '.tiff'), label_mask)
            
        else:
            pass
        
    return None

## Post Processing 
Post-processing of predicted label masks into binary masks for evaluation. The overall effect of this step is to separate touching nuclei by adding a 2-pixel gap (one-pixel erosion on each nucleus). Next, binary masks (both segmented and ground truth data) can be used to compute the F1-scores by utilizing connected component analysis. 

In [3]:
def postprocess(save_dir_seg, save_dir_bin):
    '''
    Produces binary masks for evaluation from the labelled mask output from the StarDist algorithm

    Parameters
    save_dir_seg: str
        the path to the directory containing the label masks output by the StarDist algorithm with the StarDist_segmentor function. All images should
        be in .tiff format
    save_dir_bin: str
        the path to the directory where the binary images will be saved as masks. Each tiff image in the input directory will
        have a corresponding mask in the directory provided by the user
    '''

    # iterating over all files in the input directory
    for file in os.listdir(save_dir_seg):
        # only processes tiff images
        if file[-5:]=='.tiff' or file[-4:]=='.tif':
            # Finding the pixels that are touching between any two nuclei
            label_mask = io.imread(os.path.join(save_dir_seg, file))
            boundary_bool = skimage.segmentation.find_boundaries(label_mask, connectivity=label_mask.ndim,
                                                                 mode='outer', background=0)
            # Converting these pixels to the background value in the label array
            label_mask[boundary_bool] = 0
            # Converting the label array into a binary mask of foreground (255) and background (0)
            nuclei_mask_final = np.zeros((label_mask.shape[0], label_mask.shape[1]))
            nuclei_mask_final[label_mask != 0] = 255
            nuclei_mask_final = np.uint8(nuclei_mask_final)
            
            # saving the binary mask in the save directory
            io.imsave(os.path.join(save_dir_bin, file[:file.find('.')] + '.tiff'), nuclei_mask_final)
        else:
            pass
        
    return None

## Running the segmentation pipeline

In [4]:
# Replace the strings for in_dir, save_dir_seg and save_dir_bin to paths on the local machine
in_dir = r'C:\Users\mrl\Desktop\nuclear_seg_github\nuclear_segmentation\imgs\DAPI_grayscale'
save_dir_seg = r'C:\Users\mrl\Desktop\nuclear_seg_github\imgs\StarDist_label_masks'
save_dir_bin = r'C:\Users\mrl\Desktop\nuclear_seg_github\imgs\StarDist_binary_masks'
# To create binary masks using StarDist segmentation algorithm
StarDist_segmentor(in_dir, save_dir_seg)
postprocess(save_dir_seg,save_dir_bin)

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


248740539.py (24): C:\Users\mrl\Desktop\nuclear_seg_github\nuclear_segmentation\imgs\StarDist_label_masks\6plexTonsil_DAPI_gray.tiff is a low contrast image


Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


248740539.py (24): C:\Users\mrl\Desktop\nuclear_seg_github\nuclear_segmentation\imgs\StarDist_label_masks\Bph2_45174_14597_DAPI_gray.tiff is a low contrast image


Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


248740539.py (24): C:\Users\mrl\Desktop\nuclear_seg_github\nuclear_segmentation\imgs\StarDist_label_masks\TME11_12154_44141_DAPI_gray.tiff is a low contrast image


Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


248740539.py (24): C:\Users\mrl\Desktop\nuclear_seg_github\nuclear_segmentation\imgs\StarDist_label_masks\Tonsil_7color_Scan2_17096_47011_gray.tiff is a low contrast image


Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


248740539.py (24): C:\Users\mrl\Desktop\nuclear_seg_github\nuclear_segmentation\imgs\StarDist_label_masks\VISTA4_47265_15281_DAPI_gray.tiff is a low contrast image
